In [1]:
import hail as hl
import bokeh
from hail.plot import show, output_notebook
hl.init(app_name="hg38_PCA")
output_notebook()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eriurn/submit/python37_intel/lib/python3.7/site-packages/pyspark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hdp/3.1.0.0-78/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-02-10 08:58:27 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.
2022-02-10 08:58:28 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-02-10 08:58:30 WARN  Client:69 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Running on Apache Spark version 3.1.2
SparkUI available at http://dice01-mlx.mlx:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.68-13190f0b6103
LOGGING: writing to /home/eriurn/Hail-joint-caller/notebooks/hail-20220210-0858-0.2.68-13190f0b6103.log


Loading BokehJS ...

In [ ]:
path_to_1kghg38 = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/20200217_biallelic_SNV/ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/gambian_genome_variation_project/release/20200217_biallelic_SNV/1kG.hg38.combined.vcf.bgz"
chr_map = {'1':'chr1', '2':'chr2', '3':'chr3', '4':'chr4', '5':'chr5', '6':'chr6', '7':'chr7', '8':'chr8', '9':'chr9', '10':'chr10', '11':'chr11', '12':'chr12', '13':'chr13', '14':'chr14', '15':'chr15', '16':'chr16', '17':'chr17', '18':'chr18', '19':'chr19', '20':'chr20', '21':'chr21', '22':'chr22', 'X':'chrX'}
mt= hl.import_vcf(path_to_1kghg38, reference_genome='GRCh38', contig_recoding=chr_map, min_partitions=4)
path_to_1kghg38_mt = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/20200217_biallelic_SNV/ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/gambian_genome_variation_project/release/20200217_biallelic_SNV/1kG.hg38.combined.mt"
mt.write(path_to_1kghg38_mt, overwrite=True)

In [21]:
path_to_1kghg38_mt = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/20200217_biallelic_SNV/ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/gambian_genome_variation_project/release/20200217_biallelic_SNV/1kG.hg38.combined.mt"
mt = hl.read_matrix_table(path_to_1kghg38_mt)

In [22]:
sa = hl.import_table('file:///home/eriurn/resources/Module4and9_VariantAnalysisUsingHail_KumarArcturus/resources/1kg_annotations.txt', 
                      impute=True, 
                      key='s')

2022-02-10 09:27:01 Hail: INFO: Reading table to impute column types
2022-02-10 09:27:02 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'population' as type str (imputed)
  Loading field 'super_population' as type str (imputed)
  Loading field 'is_female' as type bool (imputed)
  Loading field 'purple_hair' as type bool (imputed)
  Loading field 'caffeine_consumption' as type float64 (imputed)
  Loading field 'six_toes' as type bool (imputed)


In [23]:
mt = mt.annotate_cols(pheno = sa[mt.s])

In [24]:
mt = hl.sample_qc(mt)

In [25]:
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.95)

In [26]:
mt = hl.variant_qc(mt)

In [27]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

In [28]:
mt = mt.filter_rows(hl.agg.call_stats(mt.GT, mt.alleles).AF[1] > 0.01)

In [30]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, compute_loadings=True, k=10)

2022-02-10 09:28:48 Hail: INFO: hwe_normalized_pca: running PCA using 17819003 variants.
2022-02-10 09:29:51 Hail: INFO: pca: running PCA with 10 components...+ 1) / 23]
2022-02-10 09:42:29 Hail: INFO: Coerced sorted dataset=======>    (21 + 2) / 23]


In [31]:
mt = mt.annotate_rows(af=hl.agg.mean(mt.GT.n_alt_alleles()) / 2)
pca_loadings = pca_loadings.annotate(af=mt.rows()[pca_loadings.key].af) 

In [23]:
ht = pca_scores.select(PC1=pca_scores.scores[0],
                       PC2=pca_scores.scores[1],
                       PC3=pca_scores.scores[2],
                       PC4=pca_scores.scores[3])
ht = ht.annotate(pheno = sa[ht.s])

In [24]:
p1 = hl.plot.scatter(ht.PC1, ht.PC2, xlabel='PC1', ylabel='PC2', label=ht.pheno.super_population, size=6)
p2 = hl.plot.scatter(ht.PC1, ht.PC3, xlabel='PC1', ylabel='PC3', label=ht.pheno.super_population, size=6)
p3 = hl.plot.scatter(ht.PC2, ht.PC4, xlabel='PC2', ylabel='PC4', label=ht.pheno.super_population, size=6)

show(bokeh.layouts.gridplot([[p1], [p2], [p3]]))

2022-02-10 02:24:04 Hail: INFO: Coerced sorted dataset
2022-02-10 02:24:04 Hail: INFO: Coerced sorted dataset
2022-02-10 02:24:05 Hail: INFO: Coerced sorted dataset
2022-02-10 02:24:05 Hail: INFO: Coerced sorted dataset
2022-02-10 02:24:05 Hail: INFO: Coerced sorted dataset
2022-02-10 02:24:05 Hail: INFO: Coerced sorted dataset


In [32]:
# Perform pc_project
path_to_mgrb = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/mgrb/1000_samples.mt"
MGRB_mt = hl.read_matrix_table(path_to_mgrb)

In [26]:
# Might have to use MGRB_mt.LGT or use lgt_to_gt
MGRB_mt.describe() 
#GT = hl.experimental.lgt_to_gt(TOB_mt.LGT, TOB_mt.LA)

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
----------------------------------------
Entry fields:
    'LA': array<int32>
    'LGT': call
    'LAD': array<int32>
    'LPGT': call
    'LPL': array<int32>
    'RGQ': int32
    'END': int32
    'gvcf_info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_QUALapprox: array<int32>, 
        AS_RAW_BaseQRankSum: str, 
        AS_RAW_MQ: array<float64>, 
        AS_RAW_MQRankSum: array<tuple (
            float64, 
            int32
        )>, 
        AS_RAW_ReadPosRan

In [33]:
ht = hl.experimental.pc_project(MGRB_mt.LGT, pca_loadings.loadings, pca_loadings.af)

2022-02-10 09:43:22 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [35]:
# Check number of samples from MGRB
ht.scores[0].show()

2022-02-10 11:41:00 Hail: INFO: Coerced sorted dataset         (249 + 75) / 424]


,
s,
str,float64
"""AAAAB""",-9.37e-02
"""AAAAF""",-9.63e-02
"""AAAAI""",-9.50e-02
"""AAAAL""",-9.71e-02
"""AAAAO""",-9.57e-02
"""AAAAQ""",-9.67e-02
"""AAAAV""",-9.59e-02
"""AAABA""",-9.49e-02


In [ ]:
# Check number of samples from 1kGhg38
pca_scores.scores[0].count()

In [37]:
plot_data = ht.select(PC1=ht.scores[0],
                       PC2=ht.scores[1],
                       PC3=ht.scores[2],
                       PC4=ht.scores[3])
plot_data = plot_data.annotate(pheno = sa[plot_data.s])

In [46]:
p1 = hl.plot.scatter(plot_data.PC1, plot_data.PC2, xlabel='PC1', ylabel='PC2', label=plot_data.pheno.super_population, size=6)
show(p1)

2022-02-10 06:09:21 Hail: INFO: Coerced sorted dataset==>      (368 + 56) / 424]
2022-02-10 06:09:21 Hail: INFO: Coerced sorted dataset


In [37]:
ten_samples = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/mgrb/10_samples.mt"
ten_mt = hl.read_matrix_table(ten_samples)

In [38]:
fifty_samples = "file:///directflow/ClinicalGenomicsPipeline/dev/2021-02-04-PIPELINE-1885-All-Hail/EricData/mgrb/50_samples.mt"
fifty_mt = hl.read_matrix_table(fifty_samples)

In [39]:
join_result = ten_mt.union_cols(fifty_mt)

ValueError: 'MatrixTable.union_rows' expects all datasets to have the same columns. Datasets 0 and 1 have different columns (or possibly different order).

In [41]:
join_result.count()

(172747621, 150)

In [42]:
ten_mt.count()

(279847813, 100)

In [43]:
fifty_mt.count()

(175377406, 50)